### **Задание 1.** Создать таблицы со следующими структурами и загрузить данные из csv-файлов. 

In [ ]:
CREATE TABLE transaction (
    transaction_id INT,
    product_id INT,
    customer_id INT,
    transaction_date VARCHAR(30),
    online_order VARCHAR(30),
    order_status VARCHAR(30),
    brand VARCHAR(30),
    product_line VARCHAR(30),
    product_class VARCHAR(30),
    product_size VARCHAR(30),
    list_price DECIMAL(10, 2),
    standard_cost DECIMAL(10, 2)
);

CREATE TABLE customer (
    customer_id SERIAL PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    gender VARCHAR(30),
    dob VARCHAR(50),
    job_title VARCHAR(255),
    job_industry_category VARCHAR(50),
    wealth_segment VARCHAR(50),
    deceased_indicator VARCHAR(50),
    owns_car VARCHAR(30),
    address VARCHAR(50),
    postcode VARCHAR(30),
    state VARCHAR(30),
    country VARCHAR(30),
    property_valuation INT
);

### **Задание 2.1** Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.


In [ ]:
SELECT job_industry_category, COUNT(customer_id) AS customer_count
FROM customer
GROUP BY job_industry_category
ORDER BY customer_count DESC;

![2.1](./images/2.1.png)

### **Задание 2.2** Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.

In [ ]:
SELECT TO_CHAR(TO_DATE(t.transaction_date, 'YYYY-MM-DD'), 'YYYY-MM') AS transaction_month, c.job_industry_category, SUM(t.list_price) AS total_transaction_amount
FROM transactions t
JOIN customer c ON t.customer_id = c.customer_id
GROUP BY transaction_month, c.job_industry_category
ORDER BY transaction_month, c.job_industry_category;

![2.2](./images/2.2.png)

### **Задание 2.3** Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.

In [ ]:
SELECT t.brand, COUNT(t.transaction_id) AS online_order_count
FROM transaction t
JOIN customer c ON t.customer_id = c.customer_id
WHERE t.online_order = 'True' AND t.order_status = 'Approved' AND c.job_industry_category = 'IT'
GROUP BY t.brand
ORDER BY online_order_count DESC;

![2.3](./images/2.3.png)

### **Задание 2.4** Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат.

Первый способ - используем GROUP BY.

SELECT c.customer_id,
      SUM(t.list_price) AS total_transaction_amount,
      MAX(t.list_price) AS max_transaction_amount,
      MIN(t.list_price) AS min_transaction_amount,
      COUNT(t.transaction_id) AS transaction_count
FROM transaction t
JOIN customer c ON t.customer_id = c.customer_id
GROUP BY c.customer_id
ORDER BY total_transaction_amount DESC,
        transaction_count DESC;

![2.4.1](./images/2.4.1.png)

Второй способ - используем оконные функции.

In [ ]:
SELECT DISTINCT c.customer_id,
                SUM(t.list_price) OVER (PARTITION BY c.customer_id) AS total_transaction_amount,
                MAX(t.list_price) OVER (PARTITION BY c.customer_id) AS max_transaction_amount,
                MIN(t.list_price) OVER (PARTITION BY c.customer_id) AS min_transaction_amount,
                COUNT(t.transaction_id) OVER (PARTITION BY c.customer_id) AS transaction_count
FROM transaction t
JOIN customer c ON t.customer_id = c.customer_id
ORDER BY total_transaction_amount DESC, 
      tranaction_count DESC;

![2.4.2](./images/2.4.2.png)

### **Задание 2.5** Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы..

CTE для вычисления суммы транзакций для каждого клиента.

In [ ]:
WITH customer_transaction_totals AS (
    SELECT 
        c.customer_id,
        c.first_name,
        c.last_name,
        SUM(t.list_price) AS total_transaction_amount
    FROM 
        transaction t
    JOIN 
        customer c ON t.customer_id = c.customer_id
    GROUP BY 
        c.customer_id, c.first_name, c.last_name
    HAVING 
        SUM(t.list_price) IS NOT NULL
)

Поиск клиента с минимальной суммой транзакций за весь период:

In [ ]:
SELECT
      first_name,
      last_name,
      total_transaction_amount
FROM customer_transaction_totals
WHERE
      total_transaction_amount = (
        SELECT MIN(total_transaction_amount) 
        FROM customer_transaction_totals
      );

![2.5.1](./images/2.5.1.png)

Поиск клиента с максимальной суммой транзакций за весь период

In [ ]:
SELECT
      first_name,
      last_name,
      total_transaction_amount
FROM customer_transaction_totals
WHERE
      total_transaction_amount = (
        SELECT MAX(total_transaction_amount) 
        FROM customer_transaction_totals
      );

![2.5.2](./images/2.5.2.png)

### **Задание 2.6** Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.

In [ ]:
WITH ranked_transactions AS (
    SELECT
          customer_id,
          transaction_id,
          transaction_date,
          list_price AS transaction_amount,
          ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY transaction_date) AS transaction_rank
    FROM transaction
)
SELECT
      customer_id,
      transaction_id,
      transaction_date AS first_transaction_date,
      transaction_amount AS first_transaction_amount
FROM ranked_transactions
WHERE transaction_rank = 1
ORDER BY customer_id;

![2.6](./images/2.6.png)

### **Задание 2.7** Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях).

In [ ]:
WITH tx_dates AS (
    SELECT
        customer_id,
        to_date(transaction_date, 'DD.MM.YYYY') AS tx_date
    FROM transaction
),
tx_intervals AS (
    SELECT
        customer_id,
        tx_date,
        LAG(tx_date) OVER (PARTITION BY customer_id ORDER BY tx_date) AS prev_tx_date,
        tx_date - LAG(tx_date) OVER (PARTITION BY customer_id ORDER BY tx_date) AS interval_days
    FROM tx_dates
),
max_intervals AS (
    SELECT
        customer_id,
        MAX(interval_days) AS max_interval
    FROM tx_intervals
    WHERE interval_days IS NOT NULL
    GROUP BY customer_id
)
SELECT
    c.first_name,
    c.last_name,
    c.job_title,
    mx.max_interval
FROM customer c
INNER JOIN max_intervals mx ON mx.customer_id = c.customer_id
WHERE mx.max_interval = (SELECT MAX(max_interval) FROM max_intervals);


![2.7](./images/2.7.png)